# Grep demo

In [3]:
import iknowpy

iknow = iknowpy.iKnowEngine()

In [42]:
iknow.index("The patient did not report kidney beans, but had more serious issues.","en")

In [43]:
print(iknow.m_index)

{'sentences': [{'entities': [{'type': 'NonRelevant', 'offset_start': 0, 'offset_stop': 3, 'index': 'the', 'dominance_value': 0.0, 'entity_id': 0}, {'type': 'Concept', 'offset_start': 4, 'offset_stop': 11, 'index': 'patient', 'dominance_value': 333.0, 'entity_id': 1}, {'type': 'Relation', 'offset_start': 12, 'offset_stop': 26, 'index': 'did not report', 'dominance_value': 1000.0, 'entity_id': 2}, {'type': 'Concept', 'offset_start': 27, 'offset_stop': 40, 'index': 'kidney beans', 'dominance_value': 666.0, 'entity_id': 3}, {'type': 'Relation', 'offset_start': 41, 'offset_stop': 48, 'index': 'but had', 'dominance_value': 666.0, 'entity_id': 4}, {'type': 'Concept', 'offset_start': 49, 'offset_stop': 69, 'index': 'more serious issues', 'dominance_value': 1000.0, 'entity_id': 5}], 'sent_attributes': [{'type': 'Negation', 'offset_start': 12, 'offset_stop': 19, 'marker': 'did not', 'value': '', 'unit': '', 'value2': '', 'unit2': '', 'entity_ref': 2}], 'path': [1, 2, 3, 4, 5], 'path_attributes':

In [44]:
# print basic parsing output
for s in iknow.m_index['sentences']:
    for e in s['entities']:
        print('<'+e['type']+'>'+e['index']+'</'+e['type']+'>', end=' ')
    print('\n')

<NonRelevant>the</NonRelevant> <Concept>patient</Concept> <Relation>did not report</Relation> <Concept>kidney beans</Concept> <Relation>but had</Relation> <Concept>more serious issues</Concept> 



In [47]:
# print negation output
for s in iknow.m_index['sentences']:
    
    # first build an array marking start and end positions
    pos = 0
    negated = []
    for a in s['path_attributes']:
        if a['type']=="Negation":
            #print(a)
            for p in range(pos,a['pos']-1):
                negated.append(0)
            for p in range(a['pos'],a['pos']+a['span']):
                negated.append(1)
            pos = a['pos']+a['span']
    for p in range(pos, len(s['entities'])):
        negated.append(0)
    pos = 0
    for e in s['entities']:
        if e['type'] == 'NonRelevant': continue
        if negated[pos]: print("<neg>", end='')
        print(e['index'], end='')
        if negated[pos]: print("</neg>", end='')
        print(" ", end="")
        pos = pos+1
    print('\n')
    #print(negated)

{'type': 'Negation', 'pos': 0, 'span': 3}
<neg>patient</neg> <neg>did not report</neg> <neg>kidney beans</neg> but had more serious issues 

[1, 1, 1, 0, 0, 0]


In [54]:
# now turn into a function
def strip_negative(text, language="en", iknow=iknowpy.iKnowEngine()):
    iknow.index(text,language)
    stripped = ""
    for s in iknow.m_index['sentences']:
        pos = 0
        negated = []
        for a in s['path_attributes']:
            if a['type']=="Negation":
                #print(a)
                for p in range(pos,a['pos']-1):
                    negated.append(0)
                for p in range(a['pos'],a['pos']+a['span']):
                    negated.append(1)
                pos = a['pos']+a['span']
        for p in range(pos, len(s['entities'])):
            negated.append(0)
        pos = 0
        for e in s['entities']:
            if e['type'] == 'NonRelevant': continue
            if negated[pos] != 1: 
                stripped += e['index']
                stripped+= " "
            pos = pos+1
        stripped += '\n'
    return stripped
        

print(strip_negative("he wasn't very happy with the cake, even though we put much effort into it."))

even though we put much effort into it 

